In [3]:
from llama2 import ContentHandler, MarketTrendAndInvestmentAdviseToolChain, SagemakerLLM
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationTokenBufferMemory
from dotenv import load_dotenv


load_dotenv(dotenv_path="env")

llm_gpt4 = ChatOpenAI(
    model="gpt-4",
    temperature=0.9,
    verbose=True,
)
llm_gpt35 = ChatOpenAI(
    # model="gpt-4",
    temperature=0.9,
    verbose=True,
)
content_handler = ContentHandler()
llama2_09 = SagemakerLLM(
    endpoint_name="huggingface-pytorch-tgi-inference-2023-07-24-07-23-15-934",
    # credentials_profile_name="default",
    region_name="us-east-1",
    model_kwargs={
        "parameters": {
            "do_sample": True,
            # "top_p": 0.9,
            # "top_k": 10,
            "repetition_penalty": 1.03,
            "max_new_tokens": 2048,
            "temperature": 0.9,
            "return_full_text": False,
            # "max_length":2048,
            "truncate": 2048,
            # "num_return_sequences":2000,
            # "stop": ["\n"],
        },
    },
    content_handler=content_handler,
    verbose=True,
)
mtia_tool = MarketTrendAndInvestmentAdviseToolChain.from_create(verbose=True)
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Cryptocurrency Research Report System",
        func=mtia_tool.run,
        description="You can use this tool when you need to generate the latest Cryptocurrency research report. The input should be a complete question about the request to generate the latest market research report of Cryptocurrency. This tool will generate the textual content of the market research report for you.",
        coroutine=mtia_tool.arun,
    ),
    Tool(
        name="Current Search",
        func=search.run,
        description="""
            useful for when you need to answer questions about current events or the current state of the world or you need to ask with search. 
            the input to this should be a single search term.
            """,
        coroutine=search.arun,
    ),
]
memory = ConversationTokenBufferMemory(
    llm=llm_gpt4, memory_key="chat_history", max_token_limit=3000, return_messages=True
)
agent_excutor = initialize_agent(
    tools=tools,
    llm=llm_gpt4,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs={
        "verbose": True,
    },
)


user_input = input("Your prompt: ")
await agent_excutor.arun(input=user_input)

Your prompt:  请生成一篇btc的最新研究报告




> Entering new  chain...
```json
{
    "action": "Cryptocurrency Research Report System",
    "action_input": "请生成一篇btc的最新研究报告"
}
```

> Entering new  chain...
User may ask some cryptocurrency's market trend.
Please generate a complete question in English, using user's input below.
User's input:请生成一篇btc的最新研究报告
Complete question:
User may ask some cryptocurrency's market trend.
Please generate a complete question in English, using user's input below.
User's input:请生成一篇btc的最新研究报告
Complete question:
What is the latest research report on Bitcoin's market trend?
What is the latest research report on Bitcoin's market trend?


> Entering new  chain...
The user's question may be to ask the latest market trend of a certain cryptocurrency. 
The following index tools can help users analyze market trend.
Index tool's names are RSI,CCI,DMI,PSAR,STOCHRSI,CMF
Please generats questions, to ask the latest index above of the cryptocurrency with its symbol in user's question.
Note that some index are c

'Based on the latest market trends and indicators, Bitcoin is showing a moderately strong upward trend with a slight chance of growth. However, the current PSAR value provides a resistance level, and the CMF value indicates a slight sell signal, suggesting there may be some resistance to further price increases. Please consider these factors when making investment decisions in the Bitcoin market.'